## GBP/TRY data set

In [8]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [9]:
data_GBP_TR = pd.read_csv("csv files/GBP_TRY.csv")
data_GBP_TR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2900 non-null   object 
 1   Price     2900 non-null   float64
 2   Open      2900 non-null   float64
 3   High      2900 non-null   float64
 4   Low       2900 non-null   float64
 5   Vol.      1044 non-null   object 
 6   Change %  2900 non-null   object 
dtypes: float64(4), object(3)
memory usage: 158.7+ KB


In [10]:
df = data_GBP_TR.drop(['Open','High','Low','Vol.','Change %'], axis=1)


In [11]:
df = df.rename(columns={'Date': 'Tarih', 'Price': 'Şimdi'})
df['Şimdi'] = df['Şimdi'].apply(lambda x:x/20)
df

,Tarih,Şimdi
0,12/30/2022,1.130360
1,12/29/2022,1.126505
2,12/28/2022,1.124440
3,12/27/2022,1.121975
4,12/26/2022,1.125305
...,...,...
2895,01/06/2012,0.144870
2896,01/05/2012,0.145830
2897,01/04/2012,0.146710
2898,01/03/2012,0.146525


## LSTM

In [12]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.13036  1.126505 1.12444  1.121975 1.125305] 1.125235
[1.126505 1.12444  1.121975 1.125305 1.125235] 1.12255
[1.12444  1.121975 1.125305 1.125235 1.12255 ] 1.1282450000000002
[1.121975 1.125305 1.125235 1.12255  1.128245] 1.13575
[1.125305 1.125235 1.12255  1.128245 1.13575 ] 1.132165
[1.125235 1.12255  1.128245 1.13575  1.132165] 1.1311
[1.12255  1.128245 1.13575  1.132165 1.1311  ] 1.13461
[1.128245 1.13575  1.132165 1.1311   1.13461 ] 1.156595
[1.13575  1.132165 1.1311   1.13461  1.156595] 1.147565
[1.132165 1.1311   1.13461  1.156595 1.147565] 1.143105
[1.1311   1.13461  1.156595 1.147565 1.143105] 1.142795
[1.13461  1.156595 1.147565 1.143105 1.142795] 1.13965
[1.156595 1.147565 1.143105 1.142795 1.13965 ] 1.13667
[1.147565 1.143105 1.142795 1.13965  1.13667 ] 1.130195
[1.143105 1.142795 1.13965  1.13667  1.130195] 1.134865
[1.142795 1.13965  1.13667  1.130195 1.134865] 1.14455
[1.13965  1.13667  1.130195 1.134865 1.14455 ] 1.14033
[1.13667  1.130195 1.134865 1.14455  1.14033 ] 

In [13]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
91/91 [==============================] - 2s 6ms/step - loss: 0.0176
Epoch 2/200
91/91 [==============================] - 1s 6ms/step - loss: 2.0750e-04
Epoch 3/200
91/91 [==============================] - 2s 23ms/step - loss: 1.7559e-04
Epoch 4/200
91/91 [==============================] - 3s 30ms/step - loss: 1.6580e-04
Epoch 5/200
91/91 [==============================] - 2s 24ms/step - loss: 1.6185e-04
Epoch 6/200
91/91 [==============================] - 2s 23ms/step - loss: 1.6251e-04
Epoch 7/200
91/91 [==============================] - 3s 30ms/step - loss: 1.6814e-04
Epoch 8/200
91/91 [==============================] - 2s 24ms/step - loss: 1.6803e-04
Epoch 9/200
91/91 [==============================] - 2s 25ms/step - loss: 1.9888e-04
Epoch 10/200
91/91 [==============================] - 2s 26ms/step - loss: 1.6243e-04
Epoch 11/200
91/91 [==============================] - 2s 26ms/step - loss: 2.0945e-04
Epoch 12/200
91/91 [==============================] - 3s 30ms/step - 

## Daily

In [14]:
x = datetime.datetime.now()
print("Prediction:",round(yhat[0][0]*20,3), "₺ GBP/TRY in", x.strftime('%x'))

Prediction: 22.307 ₺ GBP/TRY in 01/02/23


## Weekly

In [15]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1],df['Şimdi'][2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],df['Şimdi'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)*20
yhat_2 = float(yhat_2)*20
yhat_3 = float(yhat_3)*20
yhat_4 = float(yhat_4)*20
yhat_5 = float(yhat_5)*20


In [16]:
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  ₺ GBP/TRY")


Monday: 22.307 Tuesday : 22.05 Wednesday: 22.17 Thursday: 22.164 Friday: 22.173  ₺ GBP/TRY


## Monthly

In [17]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [18]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.13036  1.125235 1.1311   1.142795 1.14455 ] 1.1237499713897705
[1.125235 1.1311   1.142795 1.14455  1.12375 ] 1.1045500040054321
[1.1311   1.142795 1.14455  1.12375  1.10455 ] 1.0989799499511719
[1.142795 1.14455  1.12375  1.10455  1.09898 ] 1.0559250116348267
[1.14455  1.12375  1.10455  1.09898  1.055925] 1.0785900354385376
[1.12375  1.10455  1.09898  1.055925 1.07859 ] 1.0504300594329834
[1.10455  1.09898  1.055925 1.07859  1.05043 ] 1.0364899635314941
[1.09898  1.055925 1.07859  1.05043  1.03649 ] 1.0287050008773804
[1.055925 1.07859  1.05043  1.03649  1.028705] 1.0322999954223633
[1.07859  1.05043  1.03649  1.028705 1.0323  ] 0.9994649887084961
[1.05043  1.03649  1.028705 1.0323   0.999465] 1.0415849685668945
[1.03649  1.028705 1.0323   0.999465 1.041585] 1.0563499927520752
[1.028705 1.0323   0.999465 1.041585 1.05635 ] 1.0458550453186035
[1.0323   0.999465 1.041585 1.05635  1.045855] 1.0656449794769287
[0.999465 1.041585 1.05635  1.045855 1.065645] 1.0694550275802612
[1.041585 

In [19]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


Epoch 1/200
18/18 [==============================] - 1s 6ms/step - loss: 0.0552
Epoch 2/200
18/18 [==============================] - 0s 6ms/step - loss: 0.0046
Epoch 3/200
18/18 [==============================] - 0s 6ms/step - loss: 0.0013
Epoch 4/200
18/18 [==============================] - 0s 6ms/step - loss: 8.0900e-04
Epoch 5/200
18/18 [==============================] - 0s 6ms/step - loss: 6.6686e-04
Epoch 6/200
18/18 [==============================] - 0s 11ms/step - loss: 6.6623e-04
Epoch 7/200
18/18 [==============================] - 1s 34ms/step - loss: 6.0744e-04
Epoch 8/200
18/18 [==============================] - 1s 34ms/step - loss: 6.2245e-04
Epoch 9/200
18/18 [==============================] - 0s 6ms/step - loss: 6.2403e-04
Epoch 10/200
18/18 [==============================] - 0s 6ms/step - loss: 6.2132e-04
Epoch 11/200
18/18 [==============================] - 0s 6ms/step - loss: 6.0046e-04
Epoch 12/200
18/18 [==============================] - 0s 6ms/step - loss: 6.0352e-0

In [20]:
print("Next Month -> Prediction:", round(float(yhat[0][0]*20),3), "₺ GBP/TRY in", x.strftime('%B'))


Next Month -> Prediction: 22.367 ₺ GBP/TRY in January


## Yearly

In [21]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 10 == 0:  # it equals to weekly
        exchange_2.append(df['Şimdi'][i])


In [22]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.13036  1.1311   1.14455  1.10455  1.055925] 1.0504300594329834
[1.1311   1.14455  1.10455  1.055925 1.05043 ] 1.0287050008773804
[1.14455  1.10455  1.055925 1.05043  1.028705] 0.9994649887084961
[1.10455  1.055925 1.05043  1.028705 0.999465] 1.0563499927520752
[1.055925 1.05043  1.028705 0.999465 1.05635 ] 1.0656449794769287
[1.05043  1.028705 0.999465 1.05635  1.065645] 1.0875999927520752
[1.028705 0.999465 1.05635  1.065645 1.0876  ] 1.0895500183105469
[0.999465 1.05635  1.065645 1.0876   1.08955 ] 1.0308300256729126
[1.05635  1.065645 1.0876   1.08955  1.03083 ] 1.0135300159454346
[1.065645 1.0876   1.08955  1.03083  1.01353 ] 1.0588500499725342
[1.0876  1.08955 1.03083 1.01353 1.05885] 1.024804949760437
[1.08955  1.03083  1.01353  1.05885  1.024805] 0.9916750192642212
[1.03083  1.01353  1.05885  1.024805 0.991675] 0.9219949841499329
[1.01353  1.05885  1.024805 0.991675 0.921995] 0.9456949830055237
[1.05885  1.024805 0.991675 0.921995 0.945695] 0.9606500267982483
[1.024805 0.9916

In [23]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
9/9 [==============================] - 1s 6ms/step - loss: 0.0983
Epoch 2/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0205
Epoch 3/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0080
Epoch 4/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0051
Epoch 5/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0032
Epoch 6/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0019
Epoch 7/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0015
Epoch 8/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0014
Epoch 9/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0015
Epoch 10/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0015
Epoch 11/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0013
Epoch 12/300
9/9 [==============================] - 0s 6ms/step - loss: 0.0012
Epoch 13/300
9/9 [==============================] - 0s 6ms/st

In [25]:
# onluk olarak yukarı yuvarlandı.
print("Next Year -> Prediction:", round(float(yhat[0][0]*20),3) ," ₺ GBP/TL in", x.strftime('%Y') )


Next Year -> Prediction: 21.631  ₺ GBP/TL in 2023
